In [7]:
from langchain_community.chat_models import ChatOllama
from langchain.embeddings import OllamaEmbeddings

In [8]:
llm = ChatOllama(model = "llama3")
embedding = OllamaEmbeddings(model="nomic-embed-text")

# Document Loading

In [9]:
from langchain.document_loaders import CSVLoader
file_path = "parking_spots.csv"

In [10]:
import pandas as pd
# Load the CSV file using pandas for data inspection
df = pd.read_csv(file_path)

# Sort data by floor and parking number
df_sorted = df.sort_values(by=['Floor', 'Parking Number'])

# Check the first few rows to confirm sorting
print(df_sorted.head())

# Count empty parking spots on the 1st floor
first_floor_empty = df_sorted[(df_sorted['Floor'] == 1) & (df_sorted['Parking Status'] == 'Empty')]
total_empty_first_floor = len(first_floor_empty)
print(f"Total empty parking spots on the 1st floor: {total_empty_first_floor}")



    Floor  Parking Number Parking Status  Parking Type      Metadata
1       1               2         Filled  Special Need  Near to Door
42      1              43          Empty           VIP   Left Corner
43      1              44          Empty  Special Need  Near to Door
45      1              46          Empty  Special Need  Near to Door
46      1              47         Filled           VIP  Near to Door
Total empty parking spots on the 1st floor: 89


In [11]:
# Save the sorted data back to a CSV file
sorted_file_path = "sorted_parking_spots.csv"
df_sorted.to_csv(sorted_file_path, index=False)

In [12]:
from langchain_core.tools import tool
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

@tool
def empty_parking_byfloor(floor: str) -> int:
    """Fetch Empty Parking by Floor number"""
    try:
        _floor = int(floor)
    except TypeError as e:
        _floor = str(e)

    first_floor_empty = df_sorted[(df_sorted['Floor'] == _floor) & (df_sorted['Parking Status'] == 'Empty')]
    total_empty_first_floor = len(first_floor_empty)
    print(total_empty_first_floor)
    return total_empty_first_floor

# tools = [empty_parking_byfloor]
# llm_with_tools = llm.bind_tools(tools)
# tool_map = {tool.name: tool for tool in tools}

def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


from typing import List, Literal

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt import ToolNode
    

def router(state: List[BaseMessage]) -> Literal["empty_parking_byfloor", "__end__"]:
    last_message = state[-1]
    tool_calls = last_message.tool_calls if isinstance(last_message, AIMessage) else []
    if len(tool_calls):
        return "empty_parking_byfloor"
    else:
        return "__end__"


# model = ChatOllama(model="llama3", temperature=0)
model = OllamaFunctions(model="llama3", temperature=0, format="json")
model_with_tools = model.bind_tools(
    tools=[
        {
            "name": "empty_parking_byfloor",
            "description": "return fetch empty Parking by floor number",
            "parameters": {
                "type": "object",
                "properties": {
                    "floor": {
                        "type": "int",
                        "description": "floor number, " "e.g. 1",
                    }
                },
                "required": ["floor"],
            },
        }
    ],
    function_call={"name": "empty_parking_byfloor"},
)

builder = MessageGraph()

builder.add_node("oracle", model_with_tools)

tool_node = ToolNode([empty_parking_byfloor])
builder.add_node("empty_parking_byfloor", tool_node)

builder.add_edge("empty_parking_byfloor", END)
builder.set_entry_point("oracle")

builder.add_conditional_edges("oracle", router)
runnable = builder.compile()

output = runnable.invoke("fetch empty Parking by floor number 1")
print(output)

# from langchain.tools.render import format_tool_to_openai_function

# functions = [
#     format_tool_to_openai_function(f) for f in [
#         empty_parking_byfloor
#     ]
# ]

# llm = llm.bind(functions = functions)

89
[HumanMessage(content='fetch empty Parking by floor number 1', id='a0bb49af-388c-48c5-a909-58d2c6dacc07'), AIMessage(content='', id='run-f1ac4ac2-c38f-4417-b478-5e233e9547e8-0', tool_calls=[{'name': 'empty_parking_byfloor', 'args': {'floor': 1}, 'id': 'call_2dcb532e5cc740ed988e57e493f6648f'}]), ToolMessage(content='89', name='empty_parking_byfloor', id='b0780b1d-fa60-410e-b76e-b0056ffd1ec5', tool_call_id='call_2dcb532e5cc740ed988e57e493f6648f')]


In [13]:

loader = CSVLoader(file_path=sorted_file_path)
data = loader.load()

In [14]:
data[0:10]

[Document(page_content='Floor: 1\nParking Number: 2\nParking Status: Filled\nParking Type: Special Need\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 0}),
 Document(page_content='Floor: 1\nParking Number: 43\nParking Status: Empty\nParking Type: VIP\nMetadata: Left Corner', metadata={'source': 'sorted_parking_spots.csv', 'row': 1}),
 Document(page_content='Floor: 1\nParking Number: 44\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 2}),
 Document(page_content='Floor: 1\nParking Number: 46\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 3}),
 Document(page_content='Floor: 1\nParking Number: 47\nParking Status: Filled\nParking Type: VIP\nMetadata: Near to Door', metadata={'source': 'sorted_parking_spots.csv', 'row': 4}),
 Document(page_content='Floor: 1\nParking Number: 53\nParking Status

# Document Extract

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
t_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 0
)

splits = t_splitter.split_documents(data)

# Document VectorStore

In [16]:
# Method 1
from langchain.vectorstores import Chroma

presist_directory = "docs/chroma/"

vectordb = Chroma.from_documents(
    documents= splits, 
    embedding= embedding,
    persist_directory= presist_directory
)

In [17]:
# Method 2
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding= embedding
).from_loaders([loader])

/Users/iqbal/Iqbal/Study/Deeplearning.ai/Langchain/.venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


# Document Retreival

In [18]:
question = "get me empty nomal parking near to lift"

In [19]:
result = vectordb.similarity_search(question)
result

[Document(page_content='Floor: 2\nParking Number: 517\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Lift', metadata={'row': 516, 'source': 'parking_spots.csv'}),
 Document(page_content='Floor: 2\nParking Number: 517\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Lift', metadata={'row': 299, 'source': 'sorted_parking_spots.csv'}),
 Document(page_content='Floor: 2\nParking Number: 517\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Lift', metadata={'row': 299, 'source': 'sorted_parking_spots.csv'}),
 Document(page_content='Floor: 2\nParking Number: 517\nParking Status: Empty\nParking Type: Special Need\nMetadata: Near to Lift', metadata={'row': 299, 'source': 'sorted_parking_spots.csv'})]

In [20]:
from IPython.display import display, Markdown

indexResult = index.query(question, llm=llm)
display(Markdown(indexResult))

Based on the given context, there are no normal parking slots available. However, I can suggest checking for available spots near the lift on floors 2 and 5.

Here's what I found:

* Floor 2: Parking Number 609 (Special Need) is empty.
* Floor 5: Parking Number 606 (Special Need) is empty.

If you're looking for a normal parking spot, I'm afraid there aren't any available near the lift. You may want to consider other options or ask for assistance from building management or staff.

# Document QnA

In [21]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

chat_history = []

memory = ConversationBufferMemory()
qa = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=index.vectorstore.as_retriever(), 
    return_source_documents=True,
    return_generated_question=False,
    verbose=True,
)

# qa = RetrievalQA.from_chain_type(
#     llm,
#     retriever = vectordb.as_retriever(),
#     chain_type = "stuff"
# )

In [22]:
chain = qa | runnable
# chain.invoke({"question": question})

In [23]:
question = "Looking for Empty parking"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))

/Users/iqbal/Iqbal/Study/Deeplearning.ai/Langchain/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Floor: 3
Parking Number: 3
Parking Status: Empty
Parking Type: Special Need
Metadata: Near to Lift

Floor: 3
Parking Number: 70
Parking Status: Empty
Parking Type: Normal
Metadata: Right Corner

Floor: 5
Parking Number: 60
Parking Status: Empty
Parking Type: Normal
Metadata: Right Corner

Floor: 3
Parking Number: 872
Parking Status: Empty
Parking Type: Normal
Metadata: Right Corner
Human: Looking for Empty parking

> Finished chain.

> Finished chain.


Based on the provided context, I found a few empty parking spaces:

1. Parking Number: 3 (Special Need) - Parking Status: Empty
2. Parking Number: 70 (Normal) - Parking Status: Empty
3. Parking Number: 60 (Normal) - Parking Status: Empty
4. Parking Number: 872 (Normal) - Parking Status: Empty

These parking spaces are located on Floor 3 and Floor 5, with the metadata indicating their locations near or at the right corner.

In [24]:
question = "Looking for VIP Empty parking which is near to lift on floor 1"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Floor: 2
Parking Number: 1000
Parking Status: Empty
Parking Type: VIP
Metadata: Near to Lift

Floor: 1
Parking Number: 215
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 1
Parking Number: 698
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 4
Parking Number: 56
Parking Status: Empty
Parking Type: VIP
Metadata: Near to Lift
Human: Looking for VIP Empty parking which is near to lift on floor 1

> Finished chain.

> Finished chain.


I can help you with that!

According to the available information, there are two VIP empty parking spots on Floor 1 that are near the lift:

* Parking Number: 1000 (Status: Empty)
* Parking Number: 215 (Status: Empty)

Please note that Parking Number 698 is already filled, so it's not an option for you.

In [25]:
question = "Looking for VIP Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Floor: 4
Parking Number: 56
Parking Status: Empty
Parking Type: VIP
Metadata: Near to Lift

Floor: 1
Parking Number: 75
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 4
Parking Number: 80
Parking Status: Empty
Parking Type: VIP
Metadata: Near to Lift

Floor: 4
Parking Number: 304
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift
Human: Looking for VIP Empty parking which is near to lift

> Finished chain.

> Finished chain.


Based on the provided context, I found a VIP empty parking spot that is near to a lift. The details are:

* Floor: 4
* Parking Number: 56
* Parking Status: Empty
* Parking Type: VIP
* Metadata: Near to Lift

This parking spot meets all your requirements!

In [26]:
question = "Looking for Normal Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Floor: 1
Parking Number: 265
Parking Status: Empty
Parking Type: Normal
Metadata: Near to Lift

Floor: 2
Parking Number: 400
Parking Status: Filled
Parking Type: Normal
Metadata: Near to Lift

Floor: 1
Parking Number: 625
Parking Status: Empty
Parking Type: Normal
Metadata: Near to Lift

Floor: 1
Parking Number: 240
Parking Status: Empty
Parking Type: Normal
Metadata: Near to Lift
Human: Looking for Normal Empty parking which is near to lift

> Finished chain.

> Finished chain.


Based on the provided context, I can help you find a Normal Empty parking spot that is near to a lift.

Let me scan through the information... Ah-ha! I found one!

Parking Number: 265 (on Floor 1) and Parking Number: 240 (also on Floor 1) both have the status "Empty" and are classified as Normal parking. Additionally, they are labeled as being "Near to Lift".

So, you can consider either of these two options!

In [27]:
question = "Looking for Empty parking which is near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Floor: 3
Parking Number: 3
Parking Status: Empty
Parking Type: Special Need
Metadata: Near to Lift

Floor: 4
Parking Number: 304
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 1
Parking Number: 75
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 2
Parking Number: 1000
Parking Status: Empty
Parking Type: VIP
Metadata: Near to Lift
Human: Looking for Empty parking which is near to lift

> Finished chain.

> Finished chain.


According to the context, I found an empty parking spot that is near to a lift. It's on Floor 3, Parking Number 3, and it's a Special Need parking type.

In [28]:
question = "find parking spots floor 2 and 3 near to lift"
result = qa({"question": question, "chat_history":chat_history})
display(Markdown(result["answer"]))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Floor: 4
Parking Number: 304
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 3
Parking Number: 980
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 4
Parking Number: 900
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 3
Parking Number: 309
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift
Human: find parking spots floor 2 and 3 near to lift

> Finished chain.

> Finished chain.


Based on the provided context, I can help you find parking spots on Floor 2 and 3 that are near to the lift. However, since there is no information about parking spots on those floors, I must conclude that none of the existing parking spots (which are all on Floor 4) meet your request.

If you're looking for alternative solutions or would like me to assist you in finding parking spots on a different floor, please let me know!

In [29]:
question = "How many empty parking available on Floor 1?"
result = qa({"question": question, "chat_history":chat_history})
# result
# chain.invoke({"role":"user","content": question, "chat_history":chat_history})
display(Markdown(result["answer"]))
# display(Markdown(result["result"]))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Floor: 1
Parking Number: 601
Parking Status: Empty
Parking Type: Normal
Metadata: Left Corner

Floor: 1
Parking Number: 541
Parking Status: Empty
Parking Type: Normal
Metadata: Left Corner

Floor: 1
Parking Number: 420
Parking Status: Empty
Parking Type: Normal
Metadata: Right Corner

Floor: 1
Parking Number: 750
Parking Status: Empty
Parking Type: Normal
Metadata: Right Corner
Human: How many empty parking available on Floor 1?

> Finished chain.

> Finished chain.


According to the context, there are 4 empty parking spaces available on Floor 1:

* Parking Number: 601
* Parking Number: 541
* Parking Number: 420
* Parking Number: 750

All of these parking spaces have a "Parking Status" of "Empty", indicating that they are available for use.

# Agent

In [30]:
import re

def parking_byfloor(inputs: dict) -> dict:
    """Total Empty Parking Spots by Floor number"""
    print(inputs)
    # floor = inputs.get("floor", "")
    match = inputs #re.search(r'\d+', floor)
    if match:
        _floor = int(match)
    else:
        raise ValueError("No valid floor number found in the input")

    first_floor_empty = df_sorted[(df_sorted['Floor'] == _floor) & (df_sorted['Parking Status'] == 'Empty')]
    total_empty_first_floor = len(first_floor_empty)
    return {"answer":str(total_empty_first_floor)}

In [31]:
from langchain.agents import initialize_agent, Tool

# Define the tool
parking_tool = Tool(
    name="ParkingByFloor",
    func=parking_byfloor,
    description="Total number of empty parking spots by floor number"
)

In [32]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = [parking_tool]
agent = initialize_agent(
    tools, 
    llm=llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    handle_parsing_errors=True
    )

/Users/iqbal/Iqbal/Study/Deeplearning.ai/Langchain/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [33]:
def is_parking_by_floor_relevant(query):
  # Define keywords/phrases related to parking by floor
  keywords = ["how many empty parking"]
  # Check for keywords in query or retrieved documents
  for keyword in keywords:
    if keyword in query.lower():
      print("Matched")
      return True
  return False


In [34]:
def handle_query(query):
  # Analyze retrieved information or keywords in the query
  if is_parking_by_floor_relevant(query.get("question")):
    # Call ParkingByFloor tool
    return agent.run(query.get("question"))
  else:
    # Use LLM for general conversation or other tools if defined
    return qa(query)

In [35]:
question = "How many empty parking available on floor 1?" 
result = handle_query({"question": question, "chat_history":chat_history})
print(result)

/Users/iqbal/Iqbal/Study/Deeplearning.ai/Langchain/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Matched


> Entering new AgentExecutor chain...
Thought: How can I find out how many empty parking spots are available on floor 1?

Action:
```json
{
  "action": "ParkingByFloor",
  "action_input": {"floor_number": 1}
}
```

1

Observation: {'answer': '89'}
Thought:Final Answer: 89

> Finished chain.
89


In [36]:
question = "Looking for Empty parking which is near to lift"
result = handle_query({"question": question, "chat_history":chat_history})
print(result["answer"])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Floor: 3
Parking Number: 3
Parking Status: Empty
Parking Type: Special Need
Metadata: Near to Lift

Floor: 4
Parking Number: 304
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 1
Parking Number: 75
Parking Status: Filled
Parking Type: VIP
Metadata: Near to Lift

Floor: 2
Parking Number: 1000
Parking Status: Empty
Parking Type: VIP
Metadata: Near to Lift
Human: Looking for Empty parking which is near to lift

> Finished chain.

> Finished chain.
Based on the provided context, I found an empty parking spot that is near to a lift. It's located on Floor 3 with Parking Number 3 and has a status of "Empty". Additionally, it's designated as a Special Need parking type, meaning it's d